In [22]:
# Dependencies and Setup
import pandas as pd
import numpy as np
from uszipcode import SearchEngine

# Added 'census' and 'us' to the dependency list to utilize the census wrapper
from census import Census
from us import states

# import api
from config import api_key
c = Census(api_key)

In [2]:
# Naming the data points for easier reading within the loop/API request
median_income = 'B19013_001E'
employed = 'B23025_004E'
unemployed = 'B23025_005E'
poverty = 'B17001_002E'
poverty_family = 'B17012_002E'
age = 'B01002_001E'
population = 'B01003_001E'
pop_white_alone = 'B02001_002E'
pop_black_alone = 'B02001_003E'
pop_american_indian_alone = 'B02001_005E'
pop_native_hawaiian_alone = 'B02001_006E'
pop_two_or_more_races = 'B02001_007E'
pop_hispanic_origin = 'B02001_008E'
median_home_value = 'B25077_001E'
median_gross_rent = 'B25064_001E'
commute_time_pub_transit = 'B08136_007E'
commute_time_solo_auto = 'B08136_003E'
commute_time_walked = 'B08136_011E'
transit_solo_auto = 'B08301_003E'
transit_pub_transit = 'B08301_010E'
transit_walked = 'B08301_019E'
transit_other = 'B08101_041E'
ed_none = 'B15003_002E'


In [8]:
# Starting the for loop to pull in the data fields. Having some trouble naming the four dataframes using the 'year' iterable...
# Any thoughts on how to do that?
# Run the for loop and it will give you the last dataframe in the series which is '2018', currently. I've formatted the df.
# This is almost certainly more information than we need, but I wanted to include data fields that might be relevent.
# We can always shrink down the dataframes to the fields we want to focus on.
years = [2017]
joint_df = pd.DataFrame()
for year in years:
    med_income = c.acs1.state(('NAME',
                              median_income,
                              employed,
                              unemployed,
                              poverty,
                              poverty_family,
                              age,
                              population,
                              pop_white_alone,
                              pop_black_alone,
                              pop_american_indian_alone,
                              pop_native_hawaiian_alone,
                              pop_two_or_more_races,
                              pop_hispanic_origin,
                              median_home_value,
                              median_gross_rent,
                              commute_time_solo_auto,
                              commute_time_pub_transit,
                              commute_time_walked,
                              transit_solo_auto,
                              transit_pub_transit,
                               # 'year=year' is where the 'year in years' is iterated. You can replace this with whatever
                               # year you want, take it out of for loop and run.
                              transit_walked),Census.ALL,year=year)
    census_df = pd.DataFrame(med_income)
    census_df = census_df.rename(columns={
        'NAME':'State',
        'B19013_001E':'Median Income',
        'B23025_004E':'Number Employed',
        'B23025_005E':'Number Unemployed',
        'B17001_002E':'Number Poverty',
        'B17012_002E':'Number Families in Poverty',
        'B01002_001E':'Median Age',
        'B01003_001E':'Population',
        'B02001_002E':'Pop: White Only',
        'B02001_003E':'Pop: Black Only',
        'B02001_005E':'Pop: American Indian Only',
        'B02001_006E':'Pop: Native Hawaiian Only',
        'B02001_007E':'Pop: Two or More Races',
        'B02001_008E':'Pop: Hispanic Origin',
        'B25077_001E':'Median Home Value',
        'B25064_001E':'Median Gross Rent',
        'B08136_007E':'Commute Time: Solo Auto',
        'B08136_003E':'Commute Time: Public Transit',
        'B08136_011E':'Commute Time: Walking',
        'B08301_003E':'Transit: Solo Auto',
        'B08301_010E':'Transit: Public Transit',
        'B08301_019E':'Transit: Walking'
    })
    census_df['Median Income'] = census_df['Median Income'].astype(float).map("${:,.2f}".format)
    census_df['Median Home Value'] = census_df['Median Home Value'].astype(float).map("${:,.2f}".format)
    census_df['Median Gross Rent'] = census_df['Median Gross Rent'].astype(float).map("${:,.2f}".format)
    census_df['Umemployment Rate'] = (census_df['Number Unemployed']/(census_df['Number Employed']+census_df['Number Unemployed'])).astype(float).map("{:.2%}".format)
    census_df['Poverty Rate'] = (census_df['Number Poverty']/census_df['Population']).astype(float).map("{:.2%}".format)
    census_df['Commute Time: Public Transit'] = (census_df['Commute Time: Public Transit']/census_df['Population'])
    census_df['Commute Time: Solo Auto'] = (census_df['Commute Time: Solo Auto']/census_df['Population'])
    census_df['Commute Time: Walking'] = (census_df['Commute Time: Walking']/census_df['Population'])
    census_df['Transit: Solo Auto'] = (census_df['Transit: Solo Auto']/census_df['Population']).astype(float).map("{:.2%}".format)
    census_df['Transit: Public Transit'] = (census_df['Transit: Public Transit']/census_df['Population']).astype(float).map("{:.2%}".format)
    census_df['Transit: Walking'] = (census_df['Transit: Walking']/census_df['Population']).astype(float).map("{:.2%}".format)
    census_df['Number Employed'] = census_df['Number Employed'].astype(int)
    census_df['Number Unemployed'] = census_df['Number Unemployed'].astype(int)
    census_df['Number Poverty'] = census_df['Number Poverty'].astype(int)
    census_df['Number Families in Poverty'] = census_df['Number Families in Poverty'].astype(int)
    census_df['Population'] = census_df['Population'].astype(int)
    census_df['Pop: White Only'] = census_df['Pop: White Only'].astype(int)
    census_df['Pop: Black Only'] = census_df['Pop: Black Only'].astype(int)
    census_df['Pop: American Indian Only'] = census_df['Pop: American Indian Only'].astype(int)
    census_df['Pop: Native Hawaiian Only'] = census_df['Pop: Native Hawaiian Only'].astype(int)
    census_df['Pop: Two or More Races'] = census_df['Pop: Two or More Races'].astype(int)
    census_df['Pop: Hispanic Origin'] = census_df['Pop: Hispanic Origin'].astype(int)
    census_df['Pop Rate: White Only'] = (census_df['Pop: White Only']/census_df['Population']).astype(float).map("{:.2%}".format)
    census_df['Pop Rate: Black Only'] = (census_df['Pop: Black Only']/census_df['Population']).astype(float).map("{:.2%}".format)
    census_df['Pop Rate: American Indian Only'] = (census_df['Pop: American Indian Only']/census_df['Population']).astype(float).map("{:.2%}".format)
    census_df['Pop Rate: Native Hawaiian Only'] = (census_df['Pop: Native Hawaiian Only']/census_df['Population']).astype(float).map("{:.2%}".format)
    census_df['Pop Rate: Two or More Races'] = (census_df['Pop: Two or More Races']/census_df['Population']).astype(float).map("{:.2%}".format)
    census_df['Pop Rate: Hispanic Origin'] = (census_df['Pop: Hispanic Origin']/census_df['Population']).astype(float).map("{:.2%}".format)
    census_df.insert(0, 'Year', year)
    joint_df = pd.concat([census_df, joint_df])
joint_df.drop(columns=["state"], inplace=True)
joint_df.to_csv("census_data_2017.csv",encoding="utf-8",index=False)
joint_df


,Year,State,Median Income,Number Employed,Number Unemployed,Number Poverty,Number Families in Poverty,Median Age,Population,Pop: White Only,...,Transit: Public Transit,Transit: Walking,Umemployment Rate,Poverty Rate,Pop Rate: White Only,Pop Rate: Black Only,Pop Rate: American Indian Only,Pop Rate: Native Hawaiian Only,Pop Rate: Two or More Races,Pop Rate: Hispanic Origin
0,2017,Alabama,"$48,123.00",2065885,128302,802656,154370,38.9,4874747,3312718,...,0.15%,0.48%,5.85%,16.47%,67.96%,26.82%,1.37%,0.03%,1.38%,1.92%
1,2017,Alaska,"$73,181.00",344982,28267,80012,12388,34.5,739795,474856,...,0.79%,3.49%,7.57%,10.82%,64.19%,2.99%,6.68%,1.19%,1.52%,8.57%
2,2017,Arizona,"$56,581.00",3125954,192721,1018935,176062,37.7,7016270,5442417,...,0.79%,0.88%,5.81%,14.52%,77.57%,4.38%,3.31%,0.23%,6.17%,3.80%
3,2017,Arkansas,"$45,869.00",1296881,76495,478365,87890,38.1,3004279,2293307,...,0.13%,0.77%,5.57%,15.92%,76.33%,15.26%,1.59%,0.28%,3.06%,2.83%
4,2017,California,"$71,805.00",18757501,1178551,5160208,858067,36.5,39536653,23181769,...,2.33%,1.20%,5.91%,13.05%,58.63%,5.75%,14.58%,0.39%,14.95%,4.90%
5,2017,Colorado,"$69,117.00",2899926,127259,564312,91960,36.8,5607154,4720495,...,1.66%,1.38%,4.20%,10.06%,84.19%,4.09%,3.20%,0.15%,3.91%,3.45%
6,2017,Connecticut,"$74,168.00",1805023,117360,334128,55756,40.9,3588184,2723013,...,2.33%,1.38%,6.10%,9.31%,75.89%,10.61%,4.57%,0.01%,5.33%,3.27%
7,2017,Delaware,"$62,852.00",436376,24300,126986,21665,40.1,961939,662084,...,1.00%,0.99%,5.27%,13.20%,68.83%,21.87%,4.05%,0.03%,1.91%,3.04%
8,2017,District of Columbia,"$82,372.00",378780,26726,109920,15929,34.0,693972,284536,...,17.67%,6.86%,6.59%,15.84%,41.00%,45.85%,4.11%,0.04%,5.78%,2.97%
9,2017,Florida,"$52,594.00",9488742,552245,2889506,499029,42.0,20984400,15768315,...,0.77%,0.63%,5.50%,13.77%,75.14%,16.18%,2.80%,0.07%,2.87%,2.64%


In [4]:
petdata = pd.read_csv("resources/petfinder_shelters_with_full_zip.csv")
petdata = petdata.drop(columns=["address1","address2","email","phone"])
petdata.to_csv("petdata.csv",encoding="utf-8",index=False)
petdata

,city,country,id,latitude,longitude,name,state,zip
0,Kittery,US,ME118,43.0899,-70.7415,Kittery Animal Hospital and Creature Comforts LLC,ME,03904
1,Rye,US,NH81,42.9885,-70.8282,Lilac Groves Pampered Pups,NH,03870
2,Rye,US,NH140,42.9885,-70.8282,Northern New England Westie Rescue Inc,NH,03870
3,Stratham,US,NH31,43.0028,-70.9212,NHSPCA,NH,03885
4,York,US,ME158,43.1502,-70.6281,The Grateful Dog Animal Rescue,ME,03909
...,...,...,...,...,...,...,...,...
10131,Morris,US,IL717,41.3555,-88.4125,"All Those Left Behind Animal Rescue, Inc.",IL,60450
10132,Normal,US,IL806,40.5101,-88.9866,Arrow Dog Rescue,IL,61761
10133,Westville,US,IN23,41.5536,-86.8980,Independent Cat Society,IN,46391
10134,Lansing,US,MI649,42.6813,-84.5757,Purrfect Kitties,MI,48911


In [6]:
state_ids = pd.read_csv("resources/state_translate.csv")
state_ids

,State,Code
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA
5,Colorado,CO
6,Connecticut,CT
7,Delaware,DE
8,District of Columbia,DC
9,Florida,FL


In [10]:
census_merge_df = joint_df.merge(state_ids, how='inner', on='State')
census_merge_df

,Year,State,Median Income,Number Employed,Number Unemployed,Number Poverty,Number Families in Poverty,Median Age,Population,Pop: White Only,...,Transit: Walking,Umemployment Rate,Poverty Rate,Pop Rate: White Only,Pop Rate: Black Only,Pop Rate: American Indian Only,Pop Rate: Native Hawaiian Only,Pop Rate: Two or More Races,Pop Rate: Hispanic Origin,Code
0,2017,Alabama,"$48,123.00",2065885,128302,802656,154370,38.9,4874747,3312718,...,0.48%,5.85%,16.47%,67.96%,26.82%,1.37%,0.03%,1.38%,1.92%,AL
1,2017,Alaska,"$73,181.00",344982,28267,80012,12388,34.5,739795,474856,...,3.49%,7.57%,10.82%,64.19%,2.99%,6.68%,1.19%,1.52%,8.57%,AK
2,2017,Arizona,"$56,581.00",3125954,192721,1018935,176062,37.7,7016270,5442417,...,0.88%,5.81%,14.52%,77.57%,4.38%,3.31%,0.23%,6.17%,3.80%,AZ
3,2017,Arkansas,"$45,869.00",1296881,76495,478365,87890,38.1,3004279,2293307,...,0.77%,5.57%,15.92%,76.33%,15.26%,1.59%,0.28%,3.06%,2.83%,AR
4,2017,California,"$71,805.00",18757501,1178551,5160208,858067,36.5,39536653,23181769,...,1.20%,5.91%,13.05%,58.63%,5.75%,14.58%,0.39%,14.95%,4.90%,CA
5,2017,Colorado,"$69,117.00",2899926,127259,564312,91960,36.8,5607154,4720495,...,1.38%,4.20%,10.06%,84.19%,4.09%,3.20%,0.15%,3.91%,3.45%,CO
6,2017,Connecticut,"$74,168.00",1805023,117360,334128,55756,40.9,3588184,2723013,...,1.38%,6.10%,9.31%,75.89%,10.61%,4.57%,0.01%,5.33%,3.27%,CT
7,2017,Delaware,"$62,852.00",436376,24300,126986,21665,40.1,961939,662084,...,0.99%,5.27%,13.20%,68.83%,21.87%,4.05%,0.03%,1.91%,3.04%,DE
8,2017,District of Columbia,"$82,372.00",378780,26726,109920,15929,34.0,693972,284536,...,6.86%,6.59%,15.84%,41.00%,45.85%,4.11%,0.04%,5.78%,2.97%,DC
9,2017,Florida,"$52,594.00",9488742,552245,2889506,499029,42.0,20984400,15768315,...,0.63%,5.50%,13.77%,75.14%,16.18%,2.80%,0.07%,2.87%,2.64%,FL


In [12]:
census_merge_df.loc[:,'State'] = census_merge_df.loc[:,'Code']
census_merge_df

,Year,State,Median Income,Number Employed,Number Unemployed,Number Poverty,Number Families in Poverty,Median Age,Population,Pop: White Only,...,Transit: Walking,Umemployment Rate,Poverty Rate,Pop Rate: White Only,Pop Rate: Black Only,Pop Rate: American Indian Only,Pop Rate: Native Hawaiian Only,Pop Rate: Two or More Races,Pop Rate: Hispanic Origin,Code
0,2017,AL,"$48,123.00",2065885,128302,802656,154370,38.9,4874747,3312718,...,0.48%,5.85%,16.47%,67.96%,26.82%,1.37%,0.03%,1.38%,1.92%,AL
1,2017,AK,"$73,181.00",344982,28267,80012,12388,34.5,739795,474856,...,3.49%,7.57%,10.82%,64.19%,2.99%,6.68%,1.19%,1.52%,8.57%,AK
2,2017,AZ,"$56,581.00",3125954,192721,1018935,176062,37.7,7016270,5442417,...,0.88%,5.81%,14.52%,77.57%,4.38%,3.31%,0.23%,6.17%,3.80%,AZ
3,2017,AR,"$45,869.00",1296881,76495,478365,87890,38.1,3004279,2293307,...,0.77%,5.57%,15.92%,76.33%,15.26%,1.59%,0.28%,3.06%,2.83%,AR
4,2017,CA,"$71,805.00",18757501,1178551,5160208,858067,36.5,39536653,23181769,...,1.20%,5.91%,13.05%,58.63%,5.75%,14.58%,0.39%,14.95%,4.90%,CA
5,2017,CO,"$69,117.00",2899926,127259,564312,91960,36.8,5607154,4720495,...,1.38%,4.20%,10.06%,84.19%,4.09%,3.20%,0.15%,3.91%,3.45%,CO
6,2017,CT,"$74,168.00",1805023,117360,334128,55756,40.9,3588184,2723013,...,1.38%,6.10%,9.31%,75.89%,10.61%,4.57%,0.01%,5.33%,3.27%,CT
7,2017,DE,"$62,852.00",436376,24300,126986,21665,40.1,961939,662084,...,0.99%,5.27%,13.20%,68.83%,21.87%,4.05%,0.03%,1.91%,3.04%,DE
8,2017,DC,"$82,372.00",378780,26726,109920,15929,34.0,693972,284536,...,6.86%,6.59%,15.84%,41.00%,45.85%,4.11%,0.04%,5.78%,2.97%,DC
9,2017,FL,"$52,594.00",9488742,552245,2889506,499029,42.0,20984400,15768315,...,0.63%,5.50%,13.77%,75.14%,16.18%,2.80%,0.07%,2.87%,2.64%,FL


In [17]:
# census_merge_df.drop(columns = 'Code', inplace=True)
census_merge_df.rename(columns = {'State':'state'}, inplace=True)
census_merge_df

,Year,state,Median Income,Number Employed,Number Unemployed,Number Poverty,Number Families in Poverty,Median Age,Population,Pop: White Only,...,Transit: Public Transit,Transit: Walking,Umemployment Rate,Poverty Rate,Pop Rate: White Only,Pop Rate: Black Only,Pop Rate: American Indian Only,Pop Rate: Native Hawaiian Only,Pop Rate: Two or More Races,Pop Rate: Hispanic Origin
0,2017,AL,"$48,123.00",2065885,128302,802656,154370,38.9,4874747,3312718,...,0.15%,0.48%,5.85%,16.47%,67.96%,26.82%,1.37%,0.03%,1.38%,1.92%
1,2017,AK,"$73,181.00",344982,28267,80012,12388,34.5,739795,474856,...,0.79%,3.49%,7.57%,10.82%,64.19%,2.99%,6.68%,1.19%,1.52%,8.57%
2,2017,AZ,"$56,581.00",3125954,192721,1018935,176062,37.7,7016270,5442417,...,0.79%,0.88%,5.81%,14.52%,77.57%,4.38%,3.31%,0.23%,6.17%,3.80%
3,2017,AR,"$45,869.00",1296881,76495,478365,87890,38.1,3004279,2293307,...,0.13%,0.77%,5.57%,15.92%,76.33%,15.26%,1.59%,0.28%,3.06%,2.83%
4,2017,CA,"$71,805.00",18757501,1178551,5160208,858067,36.5,39536653,23181769,...,2.33%,1.20%,5.91%,13.05%,58.63%,5.75%,14.58%,0.39%,14.95%,4.90%
5,2017,CO,"$69,117.00",2899926,127259,564312,91960,36.8,5607154,4720495,...,1.66%,1.38%,4.20%,10.06%,84.19%,4.09%,3.20%,0.15%,3.91%,3.45%
6,2017,CT,"$74,168.00",1805023,117360,334128,55756,40.9,3588184,2723013,...,2.33%,1.38%,6.10%,9.31%,75.89%,10.61%,4.57%,0.01%,5.33%,3.27%
7,2017,DE,"$62,852.00",436376,24300,126986,21665,40.1,961939,662084,...,1.00%,0.99%,5.27%,13.20%,68.83%,21.87%,4.05%,0.03%,1.91%,3.04%
8,2017,DC,"$82,372.00",378780,26726,109920,15929,34.0,693972,284536,...,17.67%,6.86%,6.59%,15.84%,41.00%,45.85%,4.11%,0.04%,5.78%,2.97%
9,2017,FL,"$52,594.00",9488742,552245,2889506,499029,42.0,20984400,15768315,...,0.77%,0.63%,5.50%,13.77%,75.14%,16.18%,2.80%,0.07%,2.87%,2.64%


In [37]:
# export income data
census_income_df = census_merge_df.iloc[:,np.r_[1,2:7,15:17]]
census_income_df.to_csv("census_income_data.csv",encoding="utf-8",index=False)
census_income_df

,state,Median Income,Number Employed,Number Unemployed,Number Poverty,Number Families in Poverty,Median Home Value,Median Gross Rent
0,AL,"$48,123.00",2065885,128302,802656,154370,"$141,300.00",$750.00
1,AK,"$73,181.00",344982,28267,80012,12388,"$273,100.00","$1,201.00"
2,AZ,"$56,581.00",3125954,192721,1018935,176062,"$223,400.00","$1,020.00"
3,AR,"$45,869.00",1296881,76495,478365,87890,"$128,500.00",$711.00
4,CA,"$71,805.00",18757501,1178551,5160208,858067,"$509,400.00","$1,447.00"
5,CO,"$69,117.00",2899926,127259,564312,91960,"$348,900.00","$1,240.00"
6,CT,"$74,168.00",1805023,117360,334128,55756,"$273,100.00","$1,125.00"
7,DE,"$62,852.00",436376,24300,126986,21665,"$252,800.00","$1,086.00"
8,DC,"$82,372.00",378780,26726,109920,15929,"$607,200.00","$1,499.00"
9,FL,"$52,594.00",9488742,552245,2889506,499029,"$214,000.00","$1,128.00"


In [36]:
# export demo data
census_demo_df = census_merge_df.iloc[:,np.r_[1,7:15]]
census_demo_df.to_csv("census_demo_data.csv",encoding="utf-8",index=False)
census_demo_df

,state,Median Age,Population,Pop: White Only,Pop: Black Only,Pop: American Indian Only,Pop: Native Hawaiian Only,Pop: Two or More Races,Pop: Hispanic Origin
0,AL,38.9,4874747,3312718,1307467,66908,1581,67308,93584
1,AK,34.5,739795,474856,22107,49385,8830,11279,63390
2,AZ,37.7,7016270,5442417,307655,232517,16453,432853,266401
3,AR,38.1,3004279,2293307,458367,47885,8551,91890,85169
4,CA,36.5,39536653,23181769,2271917,5765305,153923,5911803,1938077
5,CO,36.8,5607154,4720495,229436,179251,8400,219189,193591
6,CT,40.9,3588184,2723013,380578,163924,525,191373,117503
7,DE,40.1,961939,662084,210356,38930,244,18372,29224
8,DC,34.0,693972,284536,318220,28499,302,40116,20609
9,FL,42.0,20984400,15768315,3394508,588087,13937,602032,553886


In [35]:
# export commute data
census_commute_df = census_merge_df.iloc[:,np.r_[1,17:20]]
census_commute_df.to_csv("census_commute_data.csv",encoding="utf-8",index=False)
census_commute_df

,state,Commute Time: Public Transit,Commute Time: Solo Auto,Commute Time: Walking
0,AL,8.732737,0.052743,0.052915
1,AK,NaN,NaN,NaN
2,AZ,8.366770,0.405907,0.110781
3,AR,NaN,NaN,NaN
4,CA,9.789755,1.178373,0.163830
5,CO,9.657586,0.721892,0.171208
6,CT,9.603586,1.449800,0.174000
7,DE,9.589361,0.455902,0.104362
8,DC,5.509005,6.936620,1.279072
9,FL,9.675484,0.384551,0.081226
